In [1]:
import pandas as pd
import numpy as np

from pyspark.ml.feature import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.clustering import KMeans
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer,VectorAssembler, OneHotEncoder
from pyspark.ml import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import MinMaxScaler

In [2]:
spark = SparkSession.builder.appName('BigData').getOrCreate()
spark

In [3]:
df = spark.read.csv('datasetPC2.csv',header=True, inferSchema=True)
df.toPandas()

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,160
1,50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,4,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,121
2,50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,4,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120
3,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,8,0,23.0,0.0,1.0,3.0,134
4,50005,1,0.0,Phone,1,12.0,CC,Male,NaN,3,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5625,55626,0,10.0,Computer,1,30.0,Credit Card,Male,3.0,2,Laptop & Accessory,1,Married,6,0,18.0,1.0,2.0,4.0,151
5626,55627,0,13.0,Mobile Phone,1,13.0,Credit Card,Male,3.0,5,Fashion,5,Married,6,0,16.0,1.0,2.0,NaN,225
5627,55628,0,1.0,Mobile Phone,1,11.0,Debit Card,Male,3.0,2,Laptop & Accessory,4,Married,3,1,21.0,1.0,2.0,4.0,186
5628,55629,0,23.0,Computer,3,9.0,Credit Card,Male,4.0,5,Laptop & Accessory,4,Married,4,0,15.0,2.0,2.0,9.0,179


In [5]:
df.toPandas()

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,160
1,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,8,0,23.0,0.0,1.0,3.0,134
2,50006,1,0.0,Computer,1,22.0,Debit Card,Female,3.0,5,Mobile Phone,5,Single,2,1,22.0,4.0,6.0,7.0,139
3,50012,1,11.0,Mobile Phone,1,6.0,Debit Card,Male,3.0,4,Fashion,3,Single,10,1,13.0,0.0,1.0,0.0,154
4,50013,1,0.0,Phone,1,11.0,COD,Male,2.0,3,Mobile,3,Single,2,1,13.0,2.0,2.0,2.0,134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3769,55625,0,1.0,Mobile Phone,3,12.0,UPI,Female,2.0,5,Mobile Phone,3,Single,2,0,19.0,2.0,2.0,1.0,155
3770,55626,0,10.0,Computer,1,30.0,Credit Card,Male,3.0,2,Laptop & Accessory,1,Married,6,0,18.0,1.0,2.0,4.0,151
3771,55628,0,1.0,Mobile Phone,1,11.0,Debit Card,Male,3.0,2,Laptop & Accessory,4,Married,3,1,21.0,1.0,2.0,4.0,186
3772,55629,0,23.0,Computer,3,9.0,Credit Card,Male,4.0,5,Laptop & Accessory,4,Married,4,0,15.0,2.0,2.0,9.0,179


## Pregunta 1

In [6]:
df = df.dropna()
var_cat = [nC for nC,dt in df.dtypes if dt =='string']
var_num = [nC for nC,dt in df.dtypes if dt in ['int','double']]
var_num.remove('CustomerID')
var_num.remove('Churn')

In [7]:
df[var_num].describe().toPandas()

,summary,Tenure,CityTier,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,count,3774,3774,3774,3774,3774,3774,3774,3774,3774,3774,3774,3774,3774
1,mean,8.776894541600424,1.7077371489136195,15.743773184949655,2.9811870694223637,3.754107048224695,3.0564387917329094,4.216481187069422,0.2821939586645469,15.727609962904081,1.719660837307896,2.825384207737149,4.526232114467408,164.2151563328034
2,stddev,7.678508136323304,0.9367251653058004,8.594329775245635,0.7221577868103932,1.0252331082505033,1.3897297116506901,2.562142316392126,0.4501268879096836,3.6299101675533394,1.6745076665170657,2.4675969629393997,3.3713099811171667,24.089820509405968
3,min,0.0,1,5.0,0.0,1,1,1,0,11.0,0.0,1.0,0.0,0
4,max,51.0,3,127.0,5.0,6,5,22,1,26.0,16.0,16.0,46.0,325


In [8]:
df.groupBy('Churn').count().toPandas()

,Churn,count
0,1,631
1,0,3143


## Pregunta 2

In [9]:
etapas = []
for var in var_cat:
    ## Las 2 siguientes líneas solamente están definiendo las etapas. No se están aplicando!!
    strIdx = StringIndexer(inputCol=var, outputCol=var+"_idx")
    encoder = OneHotEncoder(inputCol=var+"_idx",outputCol=var+'_ohe')
    etapas.append(strIdx)
    etapas.append(encoder)

In [10]:
df = df.dropna()
todasColumnas = [c+'_ohe' for c in var_cat]
ensamblador = VectorAssembler(inputCols=todasColumnas,outputCol='X')
etapas.append(ensamblador)
etapas

[StringIndexer_b12b206fb986,
 OneHotEncoder_f0b857bad959,
 StringIndexer_10488f26eab5,
 OneHotEncoder_4cbee5e815b4,
 StringIndexer_37554e5726fa,
 OneHotEncoder_75b272792fbc,
 StringIndexer_abf0ed09fc32,
 OneHotEncoder_42b796294d06,
 StringIndexer_e68a7733970d,
 OneHotEncoder_23bfa13cfc7a,
 VectorAssembler_2081a0ce45d5]

In [11]:
datos_cat= df.select(var_cat)
datos_cat.toPandas()

,PreferredLoginDevice,PreferredPaymentMode,Gender,PreferedOrderCat,MaritalStatus
0,Mobile Phone,Debit Card,Female,Laptop & Accessory,Single
1,Phone,Debit Card,Male,Laptop & Accessory,Single
2,Computer,Debit Card,Female,Mobile Phone,Single
3,Mobile Phone,Debit Card,Male,Fashion,Single
4,Phone,COD,Male,Mobile,Single
...,...,...,...,...,...
3769,Mobile Phone,UPI,Female,Mobile Phone,Single
3770,Computer,Credit Card,Male,Laptop & Accessory,Married
3771,Mobile Phone,Debit Card,Male,Laptop & Accessory,Married
3772,Computer,Credit Card,Male,Laptop & Accessory,Married


In [12]:
proceso = Pipeline(stages=etapas)
modelo = proceso.fit(datos_cat)
df2 = modelo.transform(datos_cat)
df2.toPandas()

,PreferredLoginDevice,PreferredPaymentMode,Gender,PreferedOrderCat,MaritalStatus,PreferredLoginDevice_idx,PreferredLoginDevice_ohe,PreferredPaymentMode_idx,PreferredPaymentMode_ohe,Gender_idx,Gender_ohe,PreferedOrderCat_idx,PreferedOrderCat_ohe,MaritalStatus_idx,MaritalStatus_ohe,X
0,Mobile Phone,Debit Card,Female,Laptop & Accessory,Single,0.0,"(1.0, 0.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",1.0,(0.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0)",1.0,"(0.0, 1.0)","(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Phone,Debit Card,Male,Laptop & Accessory,Single,2.0,"(0.0, 0.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0)",1.0,"(0.0, 1.0)","(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
2,Computer,Debit Card,Female,Mobile Phone,Single,1.0,"(0.0, 1.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",1.0,(0.0),1.0,"(0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(0.0, 1.0)","(0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,Mobile Phone,Debit Card,Male,Fashion,Single,0.0,"(1.0, 0.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,(1.0),2.0,"(0.0, 0.0, 1.0, 0.0, 0.0)",1.0,"(0.0, 1.0)","(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
4,Phone,COD,Male,Mobile,Single,2.0,"(0.0, 0.0)",3.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0,(1.0),3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)",1.0,"(0.0, 1.0)","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3769,Mobile Phone,UPI,Female,Mobile Phone,Single,0.0,"(1.0, 0.0)",4.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0)",1.0,(0.0),1.0,"(0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(0.0, 1.0)","(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ..."
3770,Computer,Credit Card,Male,Laptop & Accessory,Married,1.0,"(0.0, 1.0)",1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(1.0, 0.0)","(0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3771,Mobile Phone,Debit Card,Male,Laptop & Accessory,Married,0.0,"(1.0, 0.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(1.0, 0.0)","(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3772,Computer,Credit Card,Male,Laptop & Accessory,Married,1.0,"(0.0, 1.0)",1.0,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(1.0, 0.0)","(0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."


## Pregunta 3

In [13]:
etapas2 = []
strIdx2 = StringIndexer(inputCol='Churn',outputCol='Y')
etapas2.append(strIdx2)

In [14]:
todasColumnas = var_num
ensamblador = VectorAssembler(inputCols=todasColumnas,outputCol='X2')
etapas2.append(ensamblador)
etapas2

[StringIndexer_c199611e6890, VectorAssembler_902dc873d07b]

In [15]:
var_num

['Tenure',
 'CityTier',
 'WarehouseToHome',
 'HourSpendOnApp',
 'NumberOfDeviceRegistered',
 'SatisfactionScore',
 'NumberOfAddress',
 'Complain',
 'OrderAmountHikeFromlastYear',
 'CouponUsed',
 'OrderCount',
 'DaySinceLastOrder',
 'CashbackAmount']

In [16]:
proceso = Pipeline(stages=etapas2)
modelo = proceso.fit(df)
df3 = modelo.transform(df)
df3.toPandas()

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,...,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount,Y,X2
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,...,Single,9,1,11.0,1.0,1.0,5.0,160,1.0,"[4.0, 3.0, 6.0, 3.0, 3.0, 2.0, 9.0, 1.0, 11.0,..."
1,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,...,Single,8,0,23.0,0.0,1.0,3.0,134,1.0,"[0.0, 3.0, 15.0, 2.0, 4.0, 5.0, 8.0, 0.0, 23.0..."
2,50006,1,0.0,Computer,1,22.0,Debit Card,Female,3.0,5,...,Single,2,1,22.0,4.0,6.0,7.0,139,1.0,"[0.0, 1.0, 22.0, 3.0, 5.0, 5.0, 2.0, 1.0, 22.0..."
3,50012,1,11.0,Mobile Phone,1,6.0,Debit Card,Male,3.0,4,...,Single,10,1,13.0,0.0,1.0,0.0,154,1.0,"[11.0, 1.0, 6.0, 3.0, 4.0, 3.0, 10.0, 1.0, 13...."
4,50013,1,0.0,Phone,1,11.0,COD,Male,2.0,3,...,Single,2,1,13.0,2.0,2.0,2.0,134,1.0,"[0.0, 1.0, 11.0, 2.0, 3.0, 3.0, 2.0, 1.0, 13.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3769,55625,0,1.0,Mobile Phone,3,12.0,UPI,Female,2.0,5,...,Single,2,0,19.0,2.0,2.0,1.0,155,0.0,"[1.0, 3.0, 12.0, 2.0, 5.0, 3.0, 2.0, 0.0, 19.0..."
3770,55626,0,10.0,Computer,1,30.0,Credit Card,Male,3.0,2,...,Married,6,0,18.0,1.0,2.0,4.0,151,0.0,"[10.0, 1.0, 30.0, 3.0, 2.0, 1.0, 6.0, 0.0, 18...."
3771,55628,0,1.0,Mobile Phone,1,11.0,Debit Card,Male,3.0,2,...,Married,3,1,21.0,1.0,2.0,4.0,186,0.0,"[1.0, 1.0, 11.0, 3.0, 2.0, 4.0, 3.0, 1.0, 21.0..."
3772,55629,0,23.0,Computer,3,9.0,Credit Card,Male,4.0,5,...,Married,4,0,15.0,2.0,2.0,9.0,179,0.0,"[23.0, 3.0, 9.0, 4.0, 5.0, 4.0, 4.0, 0.0, 15.0..."


In [17]:
modelo = LogisticRegression(featuresCol= 'X2', labelCol= 'Y')
modeloAj = modelo.fit(df3)
predict = modeloAj.transform(df3)

In [18]:
predict.toPandas()

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,...,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount,Y,X2,rawPrediction,probability,prediction
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,...,11.0,1.0,1.0,5.0,160,1.0,"[4.0, 3.0, 6.0, 3.0, 3.0, 2.0, 9.0, 1.0, 11.0,...","[0.40302358262294646, -0.40302358262294646]","[0.5994138910747118, 0.40058610892528823]",0.0
1,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,...,23.0,0.0,1.0,3.0,134,1.0,"[0.0, 3.0, 15.0, 2.0, 4.0, 5.0, 8.0, 0.0, 23.0...","[-0.4925295395931899, 0.4925295395931899]","[0.3792978540751215, 0.6207021459248785]",1.0
2,50006,1,0.0,Computer,1,22.0,Debit Card,Female,3.0,5,...,22.0,4.0,6.0,7.0,139,1.0,"[0.0, 1.0, 22.0, 3.0, 5.0, 5.0, 2.0, 1.0, 22.0...","[-1.176298819463267, 1.176298819463267]","[0.23571833081084348, 0.7642816691891565]",1.0
3,50012,1,11.0,Mobile Phone,1,6.0,Debit Card,Male,3.0,4,...,13.0,0.0,1.0,0.0,154,1.0,"[11.0, 1.0, 6.0, 3.0, 4.0, 3.0, 10.0, 1.0, 13....","[1.0610298882641014, -1.0610298882641014]","[0.7428873090906593, 0.25711269090934075]",0.0
4,50013,1,0.0,Phone,1,11.0,COD,Male,2.0,3,...,13.0,2.0,2.0,2.0,134,1.0,"[0.0, 1.0, 11.0, 2.0, 3.0, 3.0, 2.0, 1.0, 13.0...","[0.326754970511562, -0.326754970511562]","[0.5809696023366091, 0.4190303976633909]",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3769,55625,0,1.0,Mobile Phone,3,12.0,UPI,Female,2.0,5,...,19.0,2.0,2.0,1.0,155,0.0,"[1.0, 3.0, 12.0, 2.0, 5.0, 3.0, 2.0, 0.0, 19.0...","[0.7317651601929072, -0.7317651601929072]","[0.6751925052516906, 0.3248074947483094]",0.0
3770,55626,0,10.0,Computer,1,30.0,Credit Card,Male,3.0,2,...,18.0,1.0,2.0,4.0,151,0.0,"[10.0, 1.0, 30.0, 3.0, 2.0, 1.0, 6.0, 0.0, 18....","[4.077037164841719, -4.077037164841719]","[0.9833251323324963, 0.0166748676675037]",0.0
3771,55628,0,1.0,Mobile Phone,1,11.0,Debit Card,Male,3.0,2,...,21.0,1.0,2.0,4.0,186,0.0,"[1.0, 1.0, 11.0, 3.0, 2.0, 4.0, 3.0, 1.0, 21.0...","[0.6329798162168929, -0.6329798162168929]","[0.6531648190974296, 0.3468351809025704]",0.0
3772,55629,0,23.0,Computer,3,9.0,Credit Card,Male,4.0,5,...,15.0,2.0,2.0,9.0,179,0.0,"[23.0, 3.0, 9.0, 4.0, 5.0, 4.0, 4.0, 0.0, 15.0...","[6.67524099703887, -6.67524099703887]","[0.9987398215866484, 0.0012601784133515759]",0.0


## Pregunta 4

In [19]:
#Ensamblar los vectores oneHot de la data categórica con las variables numéricas. 
#Normalizar los datos usando MinMax y construir un modelo de Logistic Regression. Reportar el área bajo la curva

In [20]:
etapas = []
for var in var_cat:
    ## Las 2 siguientes líneas solamente están definiendo las etapas. No se están aplicando!!
    strIdx = StringIndexer(inputCol=var, outputCol=var+"_idx")
    encoder = OneHotEncoder(inputCol=var+"_idx",outputCol=var+'_ohe')
    etapas.append(strIdx)
    etapas.append(encoder)

strIdx2 = StringIndexer(inputCol='Churn',outputCol='Y')
etapas.append(strIdx2)

todasColumnas = [c+'_ohe' for c in var_cat] + var_num
ensamblador = VectorAssembler(inputCols=todasColumnas,outputCol='X')
etapas.append(ensamblador)

etapas

[StringIndexer_3e108ff79b75,
 OneHotEncoder_b39586eedc5a,
 StringIndexer_039caeb7a201,
 OneHotEncoder_ec7e2cae9073,
 StringIndexer_76f052d3ab43,
 OneHotEncoder_ce94879fa0f4,
 StringIndexer_afe215939221,
 OneHotEncoder_6509e50c19e0,
 StringIndexer_460d773c2e7d,
 OneHotEncoder_ad1fbf79651c,
 StringIndexer_902aecd0e642,
 VectorAssembler_b660627d403b]

In [22]:
proceso = Pipeline(stages=etapas)
modelo = proceso.fit(df)
df_f = modelo.transform(df)

min_max = MinMaxScaler(inputCol='X',outputCol='X_Min_Max')
etapas.append(min_max)

In [23]:
modelo = LogisticRegression(featuresCol='X_Min_Max', labelCol='Y')
modeloAj = modelo.fit(df_f)
predicciones = modeloAj.transform(df_f)
evaluador = BinaryClassificationEvaluator(labelCol='Y')
evaluador.evaluate(predicciones)

0.8962686179586569

## Pregunta 5

In [24]:
# Ensamblar los vectores oneHot de la data categórica con las variables numéricas. 
# Normalizar los datos usando StandardScaler 
# y construir un modelo de Logistic Regression. Reportar el área bajo la curva

In [25]:
etapas = []
for var in var_cat:
    ## Las 2 siguientes líneas solamente están definiendo las etapas. No se están aplicando!!
    strIdx = StringIndexer(inputCol=var, outputCol=var+"_idx")
    encoder = OneHotEncoder(inputCol=var+"_idx",outputCol=var+'_ohe')
    etapas.append(strIdx)
    etapas.append(encoder)

strIdx2 = StringIndexer(inputCol='Churn',outputCol='Y')
etapas.append(strIdx2)

todasColumnas = [c+'_ohe' for c in var_cat] + var_num
ensamblador = VectorAssembler(inputCols=todasColumnas,outputCol='X')
etapas.append(ensamblador)
etapas

[StringIndexer_7ed729e41f93,
 OneHotEncoder_84ab58701acd,
 StringIndexer_35b051b1b84b,
 OneHotEncoder_d01ed075bc68,
 StringIndexer_b5567c748b3a,
 OneHotEncoder_d4f3ff66abbf,
 StringIndexer_a616ebe92d31,
 OneHotEncoder_561827627194,
 StringIndexer_533727cdd06f,
 OneHotEncoder_45f0c050b1b2,
 StringIndexer_b7d35c558ea5,
 VectorAssembler_a553caa9f849]

In [26]:
normalizador = StandardScaler(inputCol='X',outputCol='X_norm')
etapas.append(normalizador)
etapas

[StringIndexer_7ed729e41f93,
 OneHotEncoder_84ab58701acd,
 StringIndexer_35b051b1b84b,
 OneHotEncoder_d01ed075bc68,
 StringIndexer_b5567c748b3a,
 OneHotEncoder_d4f3ff66abbf,
 StringIndexer_a616ebe92d31,
 OneHotEncoder_561827627194,
 StringIndexer_533727cdd06f,
 OneHotEncoder_45f0c050b1b2,
 StringIndexer_b7d35c558ea5,
 VectorAssembler_a553caa9f849,
 StandardScaler_12802ce55ac1]

In [27]:
proceso = Pipeline(stages=etapas)
modelo = proceso.fit(df)
df_f2 = modelo.transform(df)
df_f2.toPandas()

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,...,PreferredPaymentMode_ohe,Gender_idx,Gender_ohe,PreferedOrderCat_idx,PreferedOrderCat_ohe,MaritalStatus_idx,MaritalStatus_ohe,Y,X,X_norm
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,...,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",1.0,(0.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0)",1.0,"(0.0, 1.0)",1.0,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(2.0004095559490556, 0.0, 2.034840079462974, 0..."
1,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,...,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0)",1.0,"(0.0, 1.0)",1.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 0.0, 2.034840079462974, 0.0, 0.0, 0.0, 0..."
2,50006,1,0.0,Computer,1,22.0,Debit Card,Female,3.0,5,...,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",1.0,(0.0),1.0,"(0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(0.0, 1.0)",1.0,"(0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","(0.0, 2.193823770444342, 2.034840079462974, 0...."
3,50012,1,11.0,Mobile Phone,1,6.0,Debit Card,Male,3.0,4,...,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,(1.0),2.0,"(0.0, 0.0, 1.0, 0.0, 0.0)",1.0,"(0.0, 1.0)",1.0,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(2.0004095559490556, 0.0, 2.034840079462974, 0..."
4,50013,1,0.0,Phone,1,11.0,COD,Male,2.0,3,...,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0)",0.0,(1.0),3.0,"(0.0, 0.0, 0.0, 1.0, 0.0)",1.0,"(0.0, 1.0)",1.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 3.6906997130170947, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3769,55625,0,1.0,Mobile Phone,3,12.0,UPI,Female,2.0,5,...,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0)",1.0,(0.0),1.0,"(0.0, 1.0, 0.0, 0.0, 0.0)",1.0,"(0.0, 1.0)",0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, ...","(2.0004095559490556, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3770,55626,0,10.0,Computer,1,30.0,Credit Card,Male,3.0,2,...,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(1.0, 0.0)",0.0,"(0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 2.193823770444342, 0.0, 2.18644349360827..."
3771,55628,0,1.0,Mobile Phone,1,11.0,Debit Card,Male,3.0,2,...,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(1.0, 0.0)",0.0,"(1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(2.0004095559490556, 0.0, 2.034840079462974, 0..."
3772,55629,0,23.0,Computer,3,9.0,Credit Card,Male,4.0,5,...,"(0.0, 1.0, 0.0, 0.0, 0.0, 0.0)",0.0,(1.0),0.0,"(1.0, 0.0, 0.0, 0.0, 0.0)",0.0,"(1.0, 0.0)",0.0,"(0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, ...","(0.0, 2.193823770444342, 0.0, 2.18644349360827..."


In [28]:
modelo = LogisticRegression(featuresCol='X_norm', labelCol='Y')
modeloAj = modelo.fit(df_f2)
predicciones = modeloAj.transform(df_f2)
evaluador = BinaryClassificationEvaluator(labelCol='Y')
evaluador.evaluate(predicciones)

0.896297863135598

## Pregunta 6

In [29]:
#Usando el mejor modelo obtenido en las preguntas anteriores, 
#crear una función que permita realizar una predicción con un nuevo/s dato/s de test.

In [31]:
def funcion(vectorNuevo):
    
    columas = (df.columns)
    fn = spark.createDataFrame(vectorNuevo, columnas) 
    df = df.union(fn)
    
    var_cat = [nC for nC,dt in df.dtypes if dt =='string']
    var_num = [nC for nC,dt in df.dtypes if dt in ['int','double']]
    var_num.remove('CustomerID')
    var_num.remove('Churn')
    
    etapas = []
    for var in var_cat:
        ## Las 2 siguientes líneas solamente están definiendo las etapas. No se están aplicando!!
        strIdx = StringIndexer(inputCol=var, outputCol=var+"_idx")
        encoder = OneHotEncoder(inputCol=var+"_idx",outputCol=var+'_ohe')
        etapas.append(strIdx)
        etapas.append(encoder)

    strIdx2 = StringIndexer(inputCol='Churn',outputCol='Y')
    etapas.append(strIdx2)

    todasColumnas = [c+'_ohe' for c in var_cat] + var_num
    ensamblador = VectorAssembler(inputCols=todasColumnas,outputCol='X')
    etapas.append(ensamblador)
    
    normalizador = StandardScaler(inputCol='X',outputCol='X_norm')
    etapas.append(normalizador)
    
    proceso = Pipeline(stages=etapas)
    modelo = proceso.fit(df)
    df_f2 = modelo.transform(df)
       
    return df_f2.toPandas()

In [ ]:
funcion(vectorNuevo)

## Pregunta 7

In [45]:
#Usando la técnica de K-means, realizar un proceso de clustering con 5 valores de k distintos (k= 2, 3, 4, 5, 6)

In [46]:
m = 100
k = 0
for i in range(2,7):
    modelo = KMeans(k = i,featuresCol='X_norm',predictionCol='Res')
    modelo2 = modelo.fit(df_f2)
    preds = modelo2.transform(df_f2)
    lista = []
    evaluador = ClusteringEvaluator(predictionCol='Res', featuresCol= 'X_norm')
    s = evaluador.evaluate(preds)
    lista.append(s)
    if s < m:
        m = s
        k = i

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "C:\ProgramData\Anaconda3\lib\site-packages\py4j\clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\ProgramData\Anaconda3\lib\socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 